In [35]:
%matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

# To list all indices, do 
# curl -XGET 'http://atlas-kibana.mwt2.org:9200/_settings?pretty=1' 
ind="jobs_archive_2016-08*,jobs_archive_2016-09*"

# one can also use the following command to see the return json data.
# curl -XGET 'http://atlas-kibana.mwt2.org:9200/jobs_archive_2016*/_search?pretty=1&size=1'

x = {
    "fields": ["pandaid","transformation","wall_time","cpuconsumptiontime","cpu_eff","corecount"],
    "query": {
        "bool": {
            "must": [{ "match" : { "jeditaskid" : 9397978 }},
                     { "match" : { "jobstatus" : "finished"}},
                     { "match" : { "transformation" : "Reco_tf.py"}}]
        }
    }
}

# DAODMerge_tf.py  Reco_tf.py

nfinished = []
ntotal = []
timerange0 = datetime.strptime("2016-09-01","%Y-%m-%d")
timerange1 = datetime.strptime("2016-10-01","%Y-%m-%d")

# timerange1-timerange0 is a timedelta object
ndays = (timerange1 - timerange0).days  
for i in range(0, ndays):
    nfinished.append(0)
    ntotal.append(0)

page = es.search(index=ind, body=x, search_type="scan", scroll="1m")
scroll_size = page['hits']['total']

db=0
nn=0
totwallt=0
totcput=0
print("%s" % "pandaid, transformation, wall_time, cpuconsumptiontime, cpu_eff, corecount")

while (scroll_size > 0):
    try:
        sid = page['_scroll_id']
        page = es.scroll(scroll_id = sid, scroll = '1m')
        scroll_size = len(page['hits']['hits'])
        for job in page['hits']['hits']:
            pandaid = job['fields']['pandaid'][0]
            transformation = job['fields']['transformation'][0]
            wallt = job['fields']['wall_time'][0]
            cput = job['fields']['cpuconsumptiontime'][0]
            cpu_eff = job['fields']['cpu_eff'][0]
            corecount = job['fields']['corecount'][0]
            
            totwallt += (wallt * corecount)
            totcput += cput
            nn += 1
            print("%d, %s, %d, %d, %f, %d" % (pandaid,transformation,wallt,cput,cpu_eff,corecount))
    except:
        continue




pandaid, transformation, wall_time, cpuconsumptiontime, cpu_eff, corecount
2989308456, Reco_tf.py, 1745, 5661, 3.244126, 8
2989308444, Reco_tf.py, 2050, 5684, 2.772683, 8
2989366907, Reco_tf.py, 14323, 38857, 2.712909, 8
2989366909, Reco_tf.py, 15612, 40909, 2.620356, 8
2989366891, Reco_tf.py, 18827, 56260, 2.988262, 8
2989366912, Reco_tf.py, 23601, 70133, 2.971611, 8
2989366898, Reco_tf.py, 18400, 52927, 2.876467, 8
2989587865, Reco_tf.py, 23541, 115873, 4.922178, 8
2989386119, Reco_tf.py, 39415, 50476, 1.280629, 8
2989386121, Reco_tf.py, 28159, 45299, 1.608686, 8
2989379137, Reco_tf.py, 38640, 255722, 6.618064, 8
2989361607, Reco_tf.py, 30605, 161658, 5.282078, 8
2989361602, Reco_tf.py, 39112, 188842, 4.828237, 8
2989386137, Reco_tf.py, 34314, 221379, 6.451565, 8
2989386142, Reco_tf.py, 29367, 186453, 6.349065, 8
2989386143, Reco_tf.py, 32840, 212072, 6.457735, 8
2989386157, Reco_tf.py, 40425, 271289, 6.710921, 8
2989379148, Reco_tf.py, 30707, 200095, 6.516267, 8
2989379150, Reco_tf.

In [37]:
print("num jobs: %d; total walltime(s)*Ncores : %d; total cpu time(s)*Ncores : %d" % (nn, totwallt, totcput))
print("%d %d %d" % (nn, totcput, totwallt))

num jobs: 182; total walltime(s)*Ncores : 41106664; total cpu time(s)*Ncores : 27303511
182 27303511 41106664
'' was not found in history, as a file, url, nor in the user namespace.
